In [2]:
import pyarrow.parquet as pq
import duckdb
import pandas as pd
import numpy as np
import glob

In [22]:
dummy_dirs = glob.glob("*/")
item_fp = dummy_dirs[0]
header_fp = dummy_dirs[1]

In [26]:
def random_datetimes(start, end, n):
    start_ts = pd.Timestamp(start).value // 10**9  # Convert to seconds
    end_ts = pd.Timestamp(end).value // 10**9
    random_timestamps = np.random.randint(start_ts, end_ts, size=n)
    return pd.to_datetime(random_timestamps, unit='s')

In [23]:
header_raw = duckdb.sql("SELECT * FROM read_parquet('" + header_fp + "*.parquet" + "') LIMIT 10").df()
header_raw.head()

,load_ts,batch_id,audit_zip,audit_internal_path,guid,event_name,provider,event_id,create_time,create_dt,channel,hash
0,2020-10-29 03:08:34,20201028-200228,2020102814-i-040171543e1bfef37-vpNVG8pyQ6qfTH9...,SURV8__EventLogCollectionAnalyzerTask_5ffcd2d3...,5ffcd2d39d8c4074a92a70579913b286,Generic WLAN Events,Netwtw04,7003,2020-10-28 04:49:22.325511,2020-10-28,System,00000051f12e1aff3062e8f805871224
1,2020-06-30 03:08:15,20200629-200227,2020062918-i-0728cf8ab79db9e58-wYAI5XnI06cNXjE...,SURV8__EventLogCollectionAnalyzerTask_17a948bc...,17a948bc118344528774901f9db65a81,Generic WLAN Events,Netwtw08,7021,2020-06-29 09:36:24.297649,2020-06-29,System,000000a037bf0e0d88def10ecdbc99c6
2,2020-07-15 03:15:47,20200714-200232,2020071418-i-040171543e1bfef37-nF4iyEPoAwbP64Y...,SURV8__EventLogCollectionAnalyzerTask_25ac78df...,25ac78df42a54f6cac4cf3babebc4900,Generic WLAN Events,Netwtw06,7021,2020-07-14 18:24:19.649814,2020-07-14,System,000000d80c8ec37d085c7fe914dfe096
3,2019-12-01 04:09:25,20191130-200224,2019113016-i-040171543e1bfef37-1jMahMj4lVkQgrt...,SURV8__EventLogCollectionAnalyzerTask_7c6d355f...,7c6d355fae5e4af9990da47d9dc28433,Unexpected Shutdown Event 41,Microsoft-Windows-Kernel-Power,41,2019-11-30 15:22:30.399667,2019-11-30,System,0000016822f5a1965fc5c91752cd753a
4,2021-01-14 04:27:38,20210113-190228,2021011309-i-0728cf8ab79db9e58-s3wszVDAIcFs83c...,SURV8__EventLogCollectionAnalyzerTask_2bca7664...,2bca76649b9c4842ac3ea38e488e5d1b,DSAExceptionEvent,DSAService,2003,2021-01-13 00:55:12.179959,2021-01-13,Application,0000019316dded789e506018f370288c


In [27]:
random_datetimes('2020-01-01', '2020-01-02', 10)

DatetimeIndex(['2020-01-01 01:56:58', '2020-01-01 00:02:17',
               '2020-01-01 19:50:36', '2020-01-01 16:34:40',
               '2020-01-01 01:30:29', '2020-01-01 23:41:58',
               '2020-01-01 17:00:35', '2020-01-01 18:19:05',
               '2020-01-01 16:12:41', '2020-01-01 17:20:34'],
              dtype='datetime64[ns]', freq=None)

In [72]:
num_rows = 100000

header = header_raw.copy()
header = header.iloc[0:0]
header['load_ts'] = random_datetimes('2020-02-01', '2021-02-01', num_rows)
header['batch_id'] = np.random.randint(0, num_rows * 10, size=num_rows)
header['audit_zip'] = np.random.randint(0, num_rows * 10, size=num_rows)
header['audit_internal_path'] = np.random.randint(0, num_rows * 10, size=num_rows)
header['guid'] = np.random.randint(0, num_rows / 10, size=num_rows)
header['event_name'] = np.random.choice(['a', 'b', 'c', 'd'], size=num_rows)
header['provider'] = np.random.choice(['a', 'b', 'c', 'd'], size=num_rows)
header['event_id'] = np.random.choice([41, 19, 0], size=num_rows, p=[0.02, 0.01, 0.97])
header['create_time'] = random_datetimes('2020-02-01', '2021-02-01', num_rows)
header['create_dt'] = header['create_time'].dt.date
header['channel'] = np.random.choice(['a', 'b', 'c', 'd'], size=num_rows)
header['hash'] = np.arange(num_rows)

header.to_parquet('eventlog_header_hist/header.parquet')

In [73]:
items_raw = duckdb.sql("SELECT * FROM read_parquet('" + item_fp + "*.parquet" + "') LIMIT 10").df()
items_raw.head()

,load_ts,batch_id,audit_zip,audit_internal_path,hash,create_time,create_dt,key_name,value
0,2020-10-29 05:04:28,20201028-200228,2020102814-i-040171543e1bfef37-vpNVG8pyQ6qfTH9...,SURV8__EventLogCollectionAnalyzerTask_5ffcd2d3...,00000051f12e1aff3062e8f805871224,2020-10-28 04:49:22.325511,2020-10-28,description,Intel(R) Dual Band Wireless-AC 3165
1,2020-10-29 05:04:28,20201028-200228,2020102814-i-040171543e1bfef37-vpNVG8pyQ6qfTH9...,SURV8__EventLogCollectionAnalyzerTask_5ffcd2d3...,00000051f12e1aff3062e8f805871224,2020-10-28 04:49:22.325511,2020-10-28,device_name,\Device\NDMP1
2,2020-06-30 03:08:34,20200629-200227,2020062918-i-0728cf8ab79db9e58-wYAI5XnI06cNXjE...,SURV8__EventLogCollectionAnalyzerTask_17a948bc...,000000a037bf0e0d88def10ecdbc99c6,2020-06-29 09:36:24.297649,2020-06-29,description,Intel(R) Wireless-AC 9260 160MHz
3,2020-06-30 03:08:34,20200629-200227,2020062918-i-0728cf8ab79db9e58-wYAI5XnI06cNXjE...,SURV8__EventLogCollectionAnalyzerTask_17a948bc...,000000a037bf0e0d88def10ecdbc99c6,2020-06-29 09:36:24.297649,2020-06-29,device_name,\Device\NDMP2
4,2020-07-15 03:15:43,20200714-200232,2020071418-i-040171543e1bfef37-nF4iyEPoAwbP64Y...,SURV8__EventLogCollectionAnalyzerTask_25ac78df...,000000d80c8ec37d085c7fe914dfe096,2020-07-14 18:24:19.649814,2020-07-14,description,Intel(R) Dual Band Wireless-AC 8265


In [74]:
key_value_raw = pd.DataFrame({
    'index': 0,
    'key_name': ['BugcheckCode', 'NotBugcheckCode1', 'NotBugcheckCode2'],
    'value': [np.nan, 'a', 'b']
})

key_value = pd.DataFrame()
for i in range(num_rows):
    key_value_raw['index'] = i
    key_value = pd.concat([key_value, key_value_raw])
key_value = key_value.set_index('index')
key_value['value'] = key_value['value'].apply(lambda x: np.random.choice([str(x) for x in np.arange(32)]) if x is np.nan else x)
key_value.head()

,key_name,value
index,,
0,BugcheckCode,21
0,NotBugcheckCode1,a
0,NotBugcheckCode2,b
1,BugcheckCode,9
1,NotBugcheckCode1,a


In [76]:
items = items_raw.copy()
items = items.iloc[0:0]

items['hash'] = np.arange(num_rows)
items['load_ts'] = header['load_ts']
items['batch_id'] = header['batch_id']
items['audit_zip'] = header['audit_zip']
items['audit_internal_path'] = header['audit_internal_path']
items['create_time'] = header['create_time']
items['create_dt'] = header['create_dt']
items = items.drop(columns=['key_name', 'value'])
items = pd.merge(items, key_value, left_on='hash', right_on='index', how='left')

items.to_parquet('eventlog_item_hist/item.parquet')

In [78]:
items.to_parquet('eventlog_item_hist/item.parquet')

In [6]:
pq.read_schema('eventlog_item_hist/item.parquet')

FileNotFoundError: [Errno 2] Failed to open local file 'eventlog_item_hist/item.parquet'. Detail: [errno 2] No such file or directory